# Let's familiarize ourselves with the ``Simulation`` object

In this tutorial we:

1. __Initialize an empty `Simulation` object__
2. __Initialize the NEST kernel__ (`Simulation.init__kernel`)
2. __Create a network__ (`Simulation.create__network`)
2. Create sessions:
    1. __Build session models__ (`Simulation.build__session__models`)
    1. __Build the list of sessions__ from session models (`Simulation.build__sessions`)
3. __Run__ the simulation
5. __Replicate__ the simulation

In [1]:
%load_ext autoreload
%autoreload 2

import nest
import yaml
from pathlib import Path
from pprint import pprint

from denest import *
import denest

PARAMS_DIR = Path('./data_tuto/params')
OUTPUT_DIR = Path('./data_tuto/output')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1- Initialize an empty Simulation object

Empty network, no kernel initialization, etc...

In [2]:
sim = Simulation(output_dir=OUTPUT_DIR)

2020-06-26 16:17:27,203 [denest.utils.validation] INFO: 'None' tree: adding empty child kernel
2020-06-26 16:17:27,215 [denest.utils.validation] INFO: 'None' tree: adding empty child simulation
2020-06-26 16:17:27,217 [denest.utils.validation] INFO: 'None' tree: adding empty child session_models
2020-06-26 16:17:27,218 [denest.utils.validation] INFO: 'None' tree: adding empty child network
2020-06-26 16:17:27,220 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input', 'output_dir': 'output', 'sessions': []}
2020-06-26 16:17:27,222 [denest.utils.validation] INFO: Object `kernel`: params: using default value for optional parameters:
{'extension_modules': [], 'nest_seed': 1}
2020-06-26 16:17:27,228 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-26 16:17:27,230 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-26 16:17:27,242 [denest.simulation] INFO:   Setting NEST kernel status...

# 2- Initialize the NEST kernel

``Simulation.init_kernel??`` for doc

In [3]:
kernel_tree = {
    'params':
        {
            'nest_seed': 10,
            'extension_modules': [],
        },
    'nest_params':
        {
            'resolution': 0.5,
            'overwrite_files': True
        },
}

In [4]:
sim.init_kernel(kernel_tree)

2020-06-26 16:17:27,850 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-26 16:17:27,852 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-26 16:17:27,866 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-26 16:17:27,870 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5, 'overwrite_files': True})`
2020-06-26 16:17:27,872 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'data_path': 'data_tuto/output/data', 'grng_seed': 11, 'rng_seeds': range(12, 13)})
2020-06-26 16:17:27,875 [denest.simulation] INFO:   Finished setting NEST kernel status
2020-06-26 16:17:27,887 [denest.simulation] INFO:   Installing external modules...
2020-06-26 16:17:27,893 [denest.simulation] INFO:   Finished installing external modules
2020-06-26 16:17:27,900 [denest.simulation] INFO: Finished initializing kernel


In [5]:
# nest_params have been passed to nest.SetKernelStatus
print(nest.GetKernelStatus('resolution'))
print(nest.GetKernelStatus('time'))

0.5
0.0


In [6]:
# The raw data will be saved in the output directory
nest.GetKernelStatus('data_path')

'data_tuto/output/data'

In [7]:
# The kernel params are saved in the simulation's tree
sim.tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  kernel:
    params:
      nest_seed: 10
      extension_modules: []
    nest_params:
      resolution: 0.5
      overwrite_files: true
  simulation:
    params:
      output_dir: data_tuto/output
      sessions: []
    nest_params: {}
  session_models:
    params: {}
    nest_params: {}
  network:
    params: {}
    nest_params: {}
  

# 3- Create a network

We build and create the same network by passing the network tree,
using the ``Simulation.create_network`` method

In [8]:
net_tree = ParamsTree.read(PARAMS_DIR/'network/network_tree.yml')

In [9]:
sim.create_network(net_tree)

2020-06-26 16:17:28,250 [denest.simulation] INFO: Building network.
2020-06-26 16:17:28,263 [denest.network] INFO: Build N=2 ``Model`` objects
2020-06-26 16:17:28,265 [denest.network] INFO: Build N=2 ``SynapseModel`` objects
2020-06-26 16:17:28,280 [denest.network] INFO: Build N=3 ``Model`` objects
2020-06-26 16:17:28,286 [denest.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.
2020-06-26 16:17:28,288 [denest.utils.validation] INFO: Object `proj_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-26 16:17:28,291 [denest.utils.validation] INFO: Object `proj_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-26 16:17:28,331 [denest.network] INFO: Build N=2 ``ProjectionModel`` objects
2020-06-26 16:17:28,346 [denest.network] INFO: Build N=3 ``TopoProjection`` objects
2020-06-26 16:17:28,350 [denest.network] INFO: Build N=2 population recorders.
2020-06-26 16:17:28,357 [denest.network] INFO: 

In [10]:
# Network object was created and can be accessed as we learnt
sim.network.layers['l1'].gid

(52,)

In [11]:
# network tree was saved and can be re-used
sim.tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  kernel:
    params:
      nest_seed: 10
      extension_modules: []
    nest_params:
      resolution: 0.5
      overwrite_files: true
  simulation:
    params:
      output_dir: data_tuto/output
      sessions: []
    nest_params: {}
  session_models:
  
  ... [134 lines] ...

        - layers:
          - input_layer
          populations: null
          model: my_spike_detector
        projection_recorders:
        - source_layers:
          - l1
          source_population: l1_exc
          target_layers:
          - l1
          target_population: l1_inh
          projection_model: proj_1_AMPA
          model: weight_recorder
      nest_params: {}
  

# 4- Create some sessions

### so we can run the network in specific conditions


Session parameters:

- ``simulation_time`` (float): Duration of the session in ms.
    (mandatory)
- ``reset_network`` (bool): If true, ``nest.ResetNetwork()`` is
    called during session initialization (default ``False``)
- ``record`` (bool): If false, the ``start_time`` field of
    recorder nodes in NEST is set to the end time of the
    session, so that no data is recorded during the session
    (default ``True``)
- ``shift_origin`` (bool): If True, the ``origin`` flag of the
    stimulation devices of all the network's ``InputLayer``
    layers is set to the start of the session during
    initialization. Useful to repeat sessions when the
    stimulators are eg spike generators.
- ``unit_changes`` (list): List describing the changes applied
    to certain units before the start of the session.
    Passed to ``Network.set_state``.
- ``synapse_changes`` (list): List describing the changes
    applied to certain synapses before the start of the session.
    Passed to ``Network.set_state``. Refer to that
    method for a description of how ``synapse_changes`` is
    formatted and interpreted. No changes happen if empty.
    (default [])


## Build session models from a tree

In [12]:
# Notice the hierarchical inheritance as before
session_models_tree = ParamsTree.read(PARAMS_DIR/'session_models.yml').children['session_models']
print(session_models_tree)

params:
  record: true
  shift_origin: true
  simulation_time: 100.0
nest_params: {}
warmup:
  params:
    record: false
  nest_params: {}
2_spikes:
  params:
    unit_changes:
    - layers:
      - input_layer
      population_name: spike_generator
      nest_params:
        spike_times:
        - 1.0
        - 10.0
  nest_params: {}
3_spikes:
  params:
    unit_changes:
    - layers:
      - input_layer
      population_name: spike_generator
      nest_params:
        spike_times:
        - 1.0
        - 10.0
        - 20.0
  nest_params: {}



In [13]:
sim.build_session_models(session_models_tree)

2020-06-26 16:17:29,002 [denest.simulation] INFO: Build N=3 session models


In [14]:
sim.tree.children['session_models']

ParamsTree(name='session_models', parent='None')
  params:
    record: true
    shift_origin: true
    simulation_time: 100.0
  nest_params: {}
  warmup:
    params:
      record: false
    nest_params: {}
  2_spikes:
    params:
      unit_changes:
      - layers:
        - input_layer
        population_name: spike_generator
  
  ... [3 lines] ...

          - 10.0
    nest_params: {}
  3_spikes:
    params:
      unit_changes:
      - layers:
        - input_layer
        population_name: spike_generator
        nest_params:
          spike_times:
          - 1.0
          - 10.0
          - 20.0
    nest_params: {}
  

## Build a list of sessions from the ``session_models`` templates

In [15]:
sessions_order =  ['warmup', '3_spikes', '2_spikes', '3_spikes']


In [16]:
sim.build_sessions(sessions_order)

2020-06-26 16:17:29,219 [denest.simulation] INFO: Build N=4 sessions
2020-06-26 16:17:29,221 [denest.session] INFO: Creating session "00_warmup"
2020-06-26 16:17:29,223 [denest.utils.validation] INFO: Object `00_warmup`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': [], 'unit_changes': []}
2020-06-26 16:17:29,224 [denest.session] INFO: Creating session "01_3_spikes"
2020-06-26 16:17:29,232 [denest.utils.validation] INFO: Object `01_3_spikes`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': []}
2020-06-26 16:17:29,234 [denest.session] INFO: Creating session "02_2_spikes"
2020-06-26 16:17:29,235 [denest.utils.validation] INFO: Object `02_2_spikes`: params: using default value for optional parameters:
{'reset_network': False, 'synapse_changes': []}
2020-06-26 16:17:29,240 [denest.session] INFO: Creating session "03_3_spikes"
2020-06-26 16:17:29,241 [denest.utils.validation] INFO: Object `03

In [17]:
# Notice the session names
sim.sessions

[Session(00_warmup, {'record': False,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': []}),
 Session(01_3_spikes, {'record': True,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'nest_params': {'spike_times': [1.0, 10.0, 20.0]},
                    'population_name': 'spike_generator'}]}),
 Session(02_2_spikes, {'record': True,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'nest_params': {'spike_times': [1.0, 10.0]},
                    'population_name': 'spike_generator'}]}),
 Session(03_3_spikes, {'record': True,
  'reset_network': False,
  'shift_origin': True,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
        

In [18]:
# The session order is saved in the simulation parameters

sim.tree.children['simulation']

ParamsTree(name='simulation', parent='None')
  params:
    output_dir: data_tuto/output
    sessions:
    - warmup
    - 3_spikes
    - 2_spikes
    - 3_spikes
  nest_params: {}
  

# 5- Run the simulation

In [19]:
print('kernel time: ', nest.GetKernelStatus('time'))

kernel time:  0.0


In [20]:
sim.run()

2020-06-26 16:17:29,483 [denest.simulation] INFO: Running 4 sessions...
2020-06-26 16:17:29,488 [denest.simulation] INFO: Running session: '00_warmup'...
2020-06-26 16:17:29,492 [denest.session] INFO: Initializing session...
2020-06-26 16:17:29,498 [denest.network.recorders] INFO:   Setting status for recorder my_multimeter_l1_l1_exc: {'start': 100.0}
2020-06-26 16:17:29,500 [denest.network.recorders] INFO:   Setting status for recorder my_spike_detector_input_layer_parrot_neuron: {'start': 100.0}
2020-06-26 16:17:29,508 [denest.network.recorders] INFO:   Setting status for recorder weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh: {'start': 100.0}
2020-06-26 16:17:29,511 [denest.session] INFO: Setting `origin` flag to `0.0` for all stimulation devices in ``InputLayers`` for session `00_warmup`
2020-06-26 16:17:29,539 [denest.session] INFO: Finished initializing session

2020-06-26 16:17:29,540 [denest.session] INFO: Running session '00_warmup' for 100 ms
2020-06-26 16:17:29,673 [denest

In [21]:
nest.GetKernelStatus('time')

400.0

### Since the initialized Simulation object was empty, we need to save the metadata

In [22]:
sim.save_metadata()

2020-06-26 16:17:30,717 [denest.simulation] INFO: Saving simulation metadata...
2020-06-26 16:17:30,718 [denest.simulation] INFO: Creating output directory: data_tuto/output
2020-06-26 16:17:30,832 [denest.simulation] INFO: Finished saving simulation metadata


In [23]:
!ls {OUTPUT_DIR}

data               git_hash           parameter_tree.yml session_times.yml


In [24]:
!cat {OUTPUT_DIR/'session_times.yml'}

00_warmup: !!python/tuple
- 0.0
- 100.0
01_3_spikes: !!python/tuple
- 100.0
- 200.0
02_2_spikes: !!python/tuple
- 200.0
- 300.0
03_3_spikes: !!python/tuple
- 300.0
- 400.0


In [25]:
!ls {OUTPUT_DIR/'data'}

my_multimeter_l1_l1_exc-203-0.dat
my_multimeter_l1_l1_exc.yml
my_spike_detector_input_layer_parrot_neuron-204-0.gdf
my_spike_detector_input_layer_parrot_neuron.yml
weight_recorder_conn_1_AMPA-l1-l1_exc-l1-l1_inh-205-0.csv
weight_recorder_conn_1_AMPA-l1-l1_exc-l1-l1_inh.yml
weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh-205-0.csv
weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh.yml


# 6- Replicate the simulation

In [26]:
params = ParamsTree.read(OUTPUT_DIR/'parameter_tree.yml')
params

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  kernel:
    params:
      nest_seed: 10
      extension_modules: []
    nest_params:
      resolution: 0.5
      overwrite_files: true
  simulation:
    params:
      output_dir: data_tuto/output
      sessions:
      - warmup
      - 3_spikes
  
  ... [168 lines] ...

        - layers:
          - input_layer
          populations: null
          model: my_spike_detector
        projection_recorders:
        - source_layers:
          - l1
          source_population: l1_exc
          target_layers:
          - l1
          target_population: l1_inh
          projection_model: proj_1_AMPA
          model: weight_recorder
      nest_params: {}
  

In [27]:
sim = denest.Simulation(params, output_dir='./data_tuto/output_replicate')
sim.run()

2020-06-26 16:17:31,632 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-26 16:17:31,633 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-26 16:17:31,634 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-26 16:17:31,651 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-26 16:17:31,654 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5, 'overwrite_files': True})`
2020-06-26 16:17:31,663 [denest.simulation] INFO:     Calling `nest.SetKernelStatus({'data_path': 'data_tuto/output_replicate/data', 'grng_seed': 11, 'rng_seeds': range(12, 13)})
2020-06-26 16:17:31,688 [denest.simulation] INFO:   Finished setting NEST kernel status
2020-06-26 16:17:31,700 [denest.simulation] INFO:   Installing external modules...
2020-06-26 16:17:31,706 [denest.simulation] INFO:   Finished installing external modules
2020-06-26 16:17:31,708 [denes

2020-06-26 16:17:33,931 [denest.session] INFO: Session '02_2_spikes' virtual running time: 100 ms
2020-06-26 16:17:33,932 [denest.session] INFO: Session '02_2_spikes' real running time: 0h:00m:00s
2020-06-26 16:17:33,934 [denest.simulation] INFO: Done running session '02_2_spikes'
2020-06-26 16:17:33,937 [denest.simulation] INFO: Running session: '03_3_spikes'...
2020-06-26 16:17:33,941 [denest.session] INFO: Initializing session...
2020-06-26 16:17:33,942 [denest.session] INFO: Setting `origin` flag to `300.0` for all stimulation devices in ``InputLayers`` for session `03_3_spikes`
2020-06-26 16:17:33,950 [denest.utils.validation] INFO: Object `Unit changes dictionary`: params: using default value for optional parameters:
{'change_type': 'constant', 'from_array': False}
2020-06-26 16:17:33,951 [denest.network.layers] INFO: Layer='input_layer', pop='spike_generator': Applying 'constant' change, param='spike_times', from single value')
2020-06-26 16:17:34,090 [denest.session] INFO: Fini

In [28]:
!ls {'./data_tuto/output_replicate'} 

data               git_hash           parameter_tree.yml session_times.yml
